In [2]:
%matplotlib

import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import warnings

rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [3]:
data_dir = Path(r'C:\Users\789\Desktop\github_local\Machine Learning\machine-learning-projects\data\astronomical-object')
feature_dir = Path(r'C:\Users\789\Desktop\github_local\Machine Learning\machine-learning-projects\build\feature')
val_dir = Path(r'C:\Users\789\Desktop\github_local\Machine Learning\machine-learning-projects\build\val')
tst_dir = Path(r'C:\Users\789\Desktop\github_local\Machine Learning\machine-learning-projects\build\tst')
sub_dir = Path(r'C:\Users\789\Desktop\github_local\Machine Learning\machine-learning-projects\build\sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'train.csv'
sample_file = sub_dir / 'sample_submission.csv'

target_col = 'type'
n_fold = 5
n_class = 19
seed = 42

In [5]:
algo_name = 'lgbcv' #lgiht gbm cv
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [6]:
df = pd.read_csv(feature_file, index_col = 0)
print(df.shape)
df.shape

(210000, 21)


(210000, 21)

In [9]:
y = df[target_col].values[:199991]
df.drop(target_col, axis = 1, inplace=True)
trn = df.iloc[:199991].values
tst = df.iloc[199991:].values
feature_name = df.columns.tolist()
print(y.shape, trn.shape, tst.shape)

(199991,) (199991, 20) (10009, 20)


In [7]:
# Stratified K-Fold Cross Validation

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [11]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))

for i, (i_trn, i_val) in enumerate(cv.split(trn_file, y),1):
  print(f'training model for CV #{i}')
  clf = lgb.LGBMClassifier(objective = 'multiclass',
                           n_estimators=1000,
                           num_leaves=64,
                           learning_rate=0.1,
                           min_child_samples=10,
                           subsample=.5,
                           subsample_freq=1,
                           colsample_bytree=.8,
                           random_state=seed,
                           n_jobs=-1)
  clf.fit(trn[i_trn], y[i_val],
          eval_set=[(trn[i_val], y[i_val])],
          eval_metric='multiclass',
          callbacks=[lgb.early_stopping(stopping_rounds=10)])
  
  p_val[i_val, :] = clf.predict_proba(trn[i_val])
  p_tst += clf.predict_proba(tst) / n_fold

TypeError: Input should have at least 1 dimension i.e. satisfy `len(x.shape) > 0`, got scalar `array(WindowsPath('C:/Users/789/Desktop/github_local/Machine Learning/machine-learning-projects/data/astronomical-object/train.csv'),
      dtype=object)` instead.